In [95]:
import pandas as pd
import numpy as np

# reads in the data
df2 = pd.read_csv('/Users/eoinmurnaghan/Desktop/video_post_processing/log_data_cleaning/output.txt', delimiter="\t", header=None)

df2['Box_Number'], df2['Coords'] = df2[0].str.split(' ', 1).str

df2['Box_Number'] = df2[0].shift(-1)

# tentatively drops the Coords column
df2 = df2.drop('Coords', axis=1)

# renames the column headers
df2.columns = ['Test_Frame', 'Box_Number']

# splits the Box_Number column by single spacing
df2['Box_Number'], df2['Coords'] = df2['Box_Number'].str.split(' ', 1).str

# creates 4 new columns by splitting the Coords column by the comma
df2[['ymin','xmin', 'ymax', 'xmax']] = df2['Coords'].str.rsplit(',', expand=True)

df2.head()

,Test_Frame,Box_Number,Coords,ymin,xmin,ymax,xmax
0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
1,2.0,3.0,NaN,NaN,NaN,NaN,NaN
2,3.0,4.0,NaN,NaN,NaN,NaN,NaN
3,4.0,5.0,NaN,NaN,NaN,NaN,NaN
4,5.0,6.0,NaN,NaN,NaN,NaN,NaN


In [96]:
#add in functionality to remove box data from frame columns
magic = df2[ df2.Test_Frame.str.contains(",") & (df2['Box_Number'].notnull()) & df2['Coords'].isnull()].index
df2.drop(magic , inplace=True)

df2.head()

,Test_Frame,Box_Number,Coords,ymin,xmin,ymax,xmax
0,1.0,2.0,NaN,NaN,NaN,NaN,NaN
1,2.0,3.0,NaN,NaN,NaN,NaN,NaN
2,3.0,4.0,NaN,NaN,NaN,NaN,NaN
3,4.0,5.0,NaN,NaN,NaN,NaN,NaN
4,5.0,6.0,NaN,NaN,NaN,NaN,NaN


In [97]:
#remove array seperator from ymin column
df2['ymin'] = df2['ymin'].astype(str).str.replace(r"[", '')

# remove array seperator from xmax column
df2['xmax'] = df2['xmax'].astype(str).str.replace(r"]", '')

# firmly drops the Coords column
df2 = df2.drop('Coords', axis=1)

# irons out the nan issue
df2.loc[df2.ymin == 'nan', 'ymin'] = np.nan
df2.loc[df2.xmax == 'nan', 'xmax'] = np.nan

# if the ymin value is NaN, change the Box_Number to NaN
df2.loc[df2.ymin == 'NaN', 'Box_Number'] = np.nan

df2.head()

,Test_Frame,Box_Number,ymin,xmin,ymax,xmax
0,1.0,2.0,NaN,NaN,NaN,NaN
1,2.0,3.0,NaN,NaN,NaN,NaN
2,3.0,4.0,NaN,NaN,NaN,NaN
3,4.0,5.0,NaN,NaN,NaN,NaN
4,5.0,6.0,NaN,NaN,NaN,NaN


In [77]:
df2.dtypes

Test_Frame    object
Box_Number    object
ymin          object
xmin          object
ymax          object
xmax          object
dtype: object

In [98]:
#add in new columns where boolean outlines if box data lies in frame column
df2['helper'] = np.where(df2['Test_Frame'].str.contains(","), 'true', 'false')

# will be our final Frame column
df2['Frame'] = df2['Test_Frame']

#restart indexing
df2 = df2.reset_index(drop=True)

df2.head()

,Test_Frame,Box_Number,ymin,xmin,ymax,xmax,helper,Frame
0,1.0,2.0,NaN,NaN,NaN,NaN,false,1.0
1,2.0,3.0,NaN,NaN,NaN,NaN,false,2.0
2,3.0,4.0,NaN,NaN,NaN,NaN,false,3.0
3,4.0,5.0,NaN,NaN,NaN,NaN,false,4.0
4,5.0,6.0,NaN,NaN,NaN,NaN,false,5.0


In [82]:
df2.dtypes

Frame         object
Box_Number    object
ymin          object
xmin          object
ymax          object
xmax          object
dtype: object

In [99]:
# for loop which stops frame number increasing when boolean is false
for i in range(1,len(df2)):
     if df2.helper[i] == 'true':
#             and df.Frame[i-1] == 381
             df2.Frame[i] = df2.Frame[i-1]
#              df.Frame[i] = df.Test_Frame[i]
             
     else:
        df2.Frame[i] = df2.Frame[i]

In [1]:
# retains the columns we want
df2 = df2[['Frame', 'Box_Number', 'ymin', 'xmin', 'ymax', 'xmax']]

df2.head(342)

NameError: name 'df2' is not defined

In [88]:
df2['Frame'] = pd.to_numeric(df2['Frame'])

df2['Box_Number'] = pd.to_numeric(df2['Box_Number'])
df2['ymin'] = pd.to_numeric(df2['ymin'])
df2['xmin'] = pd.to_numeric(df2['xmin'])
df2['ymax'] = pd.to_numeric(df2['ymax'])
df2['xmax'] = pd.to_numeric(df2['xmax'])

df2.dtypes

Frame         float64
Box_Number    float64
ymin          float64
xmin          float64
ymax          float64
xmax          float64
dtype: object

In [90]:
df3 = df2.set_index(['Frame', df2.groupby('Frame').cumcount()])[['ymin',
                                                               'xmin',
                                                               'ymax',
                                                               'xmax']].unstack().reset_index().rename_axis(None)



# drops the multilevel index
df3.columns = df3.columns.droplevel()

df3.head(343)



,,0,1,2,3,0,1,2,3,0,1,2,3,0,1,2,3
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
df3.rename(columns={'xmin':'ymin1','xmin':'ymin2','ymax':'ymin3', 'xmax':'ymin4'}, inplace=True)
df3

,Frame,ymin,ymin2,ymin3,ymin4,ymin,ymin2,ymin3,ymin4,ymin,ymin2,ymin3,ymin4,ymin,ymin2,ymin3,ymin4
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
# outputs the data as csv
df3.to_csv('output_acceleration_new.csv', index=False)

In [43]:
df2.head()

,Frame,Box_Number,ymin,xmin,ymax,xmax,Box2,ymin2,xmin2,Box3,ymin3,xmin3,Box4,ymin4,xmin4
0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df2

,Frame,Box_Number,ymin,xmin,ymax,xmax
0,1.0,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN
5,6.0,NaN,NaN,NaN,NaN,NaN
6,7.0,NaN,NaN,NaN,NaN,NaN
7,8.0,NaN,NaN,NaN,NaN,NaN
8,9.0,NaN,NaN,NaN,NaN,NaN
9,10.0,NaN,NaN,NaN,NaN,NaN


In [128]:
# box_data['Frame'].value_counts()
df2['freq'] = df2.groupby('Frame')['Frame'].transform('count')

df2.loc[df2['freq'] == 3]

,Frame,Box_Number,ymin,xmin,ymax,xmax,freq
294,293.0,15,0.4211374521255493,0.7721205353736877,0.47053420543670654,0.8157747387886047,3
295,293.0,16,0.34510859847068787,0.41286951303482056,0.3856203258037567,0.44142651557922363,3
296,293.0,17,0.40339645743370056,0.27880188822746277,0.4638752043247223,0.3261304199695587,3
320,307.0,41,0.38596421480178833,0.6948259472846985,0.4292398691177368,0.7415624260902405,3
321,307.0,42,0.38150864839553833,0.33020395040512085,0.43013495206832886,0.37502384185791016,3
322,307.0,43,0.34640228748321533,0.4129987061023712,0.3860595226287842,0.44242140650749207,3
323,308.0,44,0.3851180970668793,0.31812310218811035,0.4284966289997101,0.3606438636779785,3
324,308.0,45,0.38894766569137573,0.7115154266357422,0.4363793134689331,0.760066032409668,3
325,308.0,46,0.34957289695739746,0.41288840770721436,0.38855141401290894,0.4415343403816223,3
330,312.0,51,0.42514511942863464,0.7712064385414124,0.4770726263523102,0.8155751824378967,3
